# 104人力銀行網站爬蟲練習
- 從104人力銀行網站爬取求職公司資訊

In [1]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import trange

### 作業流程：
1. 首先進入104人力銀行網站 https://www.104.com.tw/cust/list/index/
2. 在"地區"及"產業"下拉式選單中，選擇你希望的選項，然後按下"找公司"按鈕。
3. 取得新開啟的網址，存為 `job_104_url`
4. 模擬瀏覽器下滑動作載入更多公司資訊，全部存到 `company_info_list`

In [2]:
base_url = 'https://www.104.com.tw/company/search/?area=6001001000&indcat=1001002000&jobsource=index_cs&mode=s&page='
job_104_url = base_url + str(1)

### 開始爬取搜尋結果
- 實作下滑網頁載入更多資料
- 取得資料

In [3]:
# 載入第一頁
opt = webdriver.ChromeOptions()
browser = webdriver.Remote(command_executor='http://selenium:4444/wd/hub', options=opt)
browser.maximize_window()
browser.get(job_104_url)
time.sleep(2)  # delay一段時間等待網頁更新完成

In [4]:
# 藉由按下分頁按鈕，取得共有幾頁
pages = browser.find_element(By.XPATH,'//div[contains(@class, "company-list-filter__select-pagination")]')
pages.click()
time.sleep(0.5)
page_lists = pages.find_elements(By.XPATH,'.//li[@class="multiselect__element position-relative"]')

# 開始逐頁爬取公司資訊
company_info_list = []

for i in trange(len(page_lists), desc='頁面'):
    if i > 0:
        job_104_url = f'{base_url}{i+1}'
        browser.get(job_104_url)
        time.sleep(2)
        
    company_list  = browser.find_elements(By.CSS_SELECTOR, 'div.company-list.company-lists__item')
    for i in company_list:
        company_name = i.find_element(By.CSS_SELECTOR, 'a.company-name-link--pc').text
        company_desc = i.find_element(By.CSS_SELECTOR, 'div.company-list__description').text
        company_info_list.append((company_name, company_desc))

print(f'共找到{len(company_info_list)}間公司')

頁面: 100%|██████████| 16/16 [01:07<00:00,  4.21s/it]

共找到282間公司


In [5]:
browser.quit()

In [6]:
import pandas as pd

df = pd.DataFrame(company_info_list, columns=["name", "desc"])
print(df.shape)
df.head()

(282, 2)


,name,desc
0,德勝科技股份有限公司,德勝科技成立於民國70年，為股票上櫃公司(股票代號8048)，主要經營業務： 光纖網路局端、...
1,葳訊企業股份有限公司,成立於民國83年，民國102年12月改制為葳訊企業股份有限公司，從事行動電話和相關配件之銷售...
2,嘉信資訊科技股份有限公司,嘉信資訊科技股份有限公司 創立於民國 85 年，主要從事通訊網路科技業；擁有為數不少的客戶群...
3,宏康智慧股份有限公司,公司成立宗旨 宏康智慧是鴻海集團亞太電信與在電信資訊服務深耕25年之康維爾資訊在103年共同...
4,香港商尚域投資有限公司台灣分公司,關於vivo vivo為一個專注於智慧手機領域的手機領導品牌，vivo和追求樂趣、充滿活力、...
